In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Georgetown,MY,5.41,100.34,80.60,94,20,4.70,few clouds,0.00,0.0
1,1,Fairbanks,US,64.84,-147.72,54.00,93,90,13.87,light rain,0.38,0.0
2,2,Kaitangata,NZ,-46.28,169.85,34.11,88,1,3.83,clear sky,0.00,0.0
3,3,Nikolskoye,RU,59.70,30.79,62.60,51,0,8.95,clear sky,0.00,0.0
4,4,Rikitea,PF,-23.12,-134.97,70.99,69,92,16.73,overcast clouds,0.00,0.0


### Ask User for their min temperature preference

In [3]:
# Ask the customer to add a minimum temperature value.
while True:
    try:
        min_temp = float(input("What is the Minimum temperature you would like for your trip? "))
        break
    except ValueError:
        print('Please enter a valid number')

What is the Minimum temperature you would like for your trip? 65


### Ask User for their max temperature preference

In [4]:
# Ask the customer to add a maximum temperature value.
while True:
    try:
        max_temp = float(input("What is the Maximum temperature you would like for your trip? "))
        break
    except ValueError:
        print('Please enter a valid number')

What is the Maximum temperature you would like for your trip? 80


### Ask User if they want it to rain

In [5]:
# Ask the customer whether they would like it to be raining
while True:
    has_rain = input("Do you want it to be raining? (yes/no) ")
    if has_rain.lower() in ['yes', 'no']:
        has_rain = True if has_rain.lower() == 'yes' else False
        break
    print('Please enter either "yes" or "no"')

Do you want it to be raining? (yes/no) no


### Ask User if they want it to snow

In [6]:
# Ask the customer whether they would like it to be raining
while True:
    has_snow = input("Do you want it to be snowing? (yes/no) ")
    if has_snow.lower() in ['yes', 'no']:
        has_snow = True if has_snow.lower() == 'yes' else False
        break
    print('Please enter either "yes" or "no"')

Do you want it to be snowing? (yes/no) no


### Filter cities based on user preference


In [7]:
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp)
    & (city_data_df["Max Temp"] >= min_temp)
]
if has_rain and has_snow:
    preferred_cities_df = preferred_cities_df.loc[
        (city_data_df["Rain (inches)"] > 0.0)
        & (city_data_df["Snow (inches)"] > 0.0)
    ]
elif has_rain:
    preferred_cities_df = preferred_cities_df.loc[
        (city_data_df["Rain (inches)"] > 0.0)
        & (city_data_df["Snow (inches)"] == 0.0)
    ]
    preferred_cities_df.head(100)
elif has_snow:
    preferred_cities_df = preferred_cities_df.loc[
        (city_data_df["Rain (inches)"] == 0.0)
        & (city_data_df["Snow (inches)"] > 0.0)
    ]
else:
    preferred_cities_df = preferred_cities_df.loc[
        (city_data_df["Rain (inches)"] == 0.0)
        & (city_data_df["Snow (inches)"] == 0.0)
    ]
preferred_cities_df.head(100)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
4,4,Rikitea,PF,-23.12,-134.97,70.99,69,92,16.73,overcast clouds,0.0,0.0
7,7,Avarua,CK,-21.21,-159.78,73.40,64,100,10.29,overcast clouds,0.0,0.0
9,9,Castro,BR,-24.79,-50.01,66.78,68,0,5.35,clear sky,0.0,0.0
13,13,Constanza,DE,47.66,9.18,66.99,68,7,1.54,clear sky,0.0,0.0
14,14,Jalu,LY,29.03,21.55,78.89,42,0,16.93,clear sky,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
364,364,Lagoa,PT,39.05,-27.98,66.13,85,0,18.39,clear sky,0.0,0.0
366,366,Ikwiriri,TZ,-7.96,38.97,71.15,79,1,5.44,clear sky,0.0,0.0
368,368,Siderno,IT,38.28,16.30,73.00,78,5,8.01,clear sky,0.0,0.0
369,369,Labuhan,ID,-6.88,112.21,77.59,87,100,4.16,overcast clouds,0.0,0.0


In [8]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

### Create dataframe to store hotel names with hotel name, city, country, description

In [9]:
hotel_df = preferred_cities_df[[
    "City"
    , "Country"
    , "Max Temp"
    , "Current Description"
    , "Lat"
    , "Lng"
]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,70.99,overcast clouds,-23.12,-134.97,
7,Avarua,CK,73.40,overcast clouds,-21.21,-159.78,
9,Castro,BR,66.78,clear sky,-24.79,-50.01,
13,Constanza,DE,66.99,clear sky,47.66,9.18,
14,Jalu,LY,78.89,clear sky,29.03,21.55,
20,Faanui,PF,78.28,overcast clouds,-16.48,-151.75,
21,Dongsheng,CN,65.95,overcast clouds,39.82,109.98,
22,Traverse City,US,75.20,broken clouds,44.76,-85.62,
24,Tapes,BR,71.87,scattered clouds,-30.67,-51.40,
27,Vaini,TO,75.20,broken clouds,-21.20,-175.20,


In [10]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass
hotel_df.head(50)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,70.99,overcast clouds,-23.12,-134.97,Pension Maro'i
7,Avarua,CK,73.40,overcast clouds,-21.21,-159.78,Paradise Inn
9,Castro,BR,66.78,clear sky,-24.79,-50.01,CHACARA BAILLY
13,Constanza,DE,66.99,clear sky,47.66,9.18,Hotel Gästehaus Centro
14,Jalu,LY,78.89,clear sky,29.03,21.55,Jalu Hotel
20,Faanui,PF,78.28,overcast clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
21,Dongsheng,CN,65.95,overcast clouds,39.82,109.98,Crowne Plaza Ordos
22,Traverse City,US,75.20,broken clouds,44.76,-85.62,Bayshore Resort
24,Tapes,BR,71.87,scattered clouds,-30.67,-51.40,Cabanas Camping Union
27,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(
    locations
    , info_box_content=hotel_info
)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,70.99,overcast clouds,-23.12,-134.97,Pension Maro'i
7,Avarua,CK,73.40,overcast clouds,-21.21,-159.78,Paradise Inn
9,Castro,BR,66.78,clear sky,-24.79,-50.01,CHACARA BAILLY
13,Constanza,DE,66.99,clear sky,47.66,9.18,Hotel Gästehaus Centro
14,Jalu,LY,78.89,clear sky,29.03,21.55,Jalu Hotel
20,Faanui,PF,78.28,overcast clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
21,Dongsheng,CN,65.95,overcast clouds,39.82,109.98,Crowne Plaza Ordos
22,Traverse City,US,75.20,broken clouds,44.76,-85.62,Bayshore Resort
24,Tapes,BR,71.87,scattered clouds,-30.67,-51.40,Cabanas Camping Union
27,Vaini,TO,75.20,broken clouds,-21.20,-175.20,Keleti Beach Resort
